# Libary Imports

In [1]:
import sys

sys.path.append("../utils")
from spark_functions import create_SparkSession
from logging_functions import inject_logging
from string_functions import to_snake_case

spark = create_SparkSession()

# Load Data
## Camera Traffic Counts

In [7]:
base_path = "../data"


camera_traffic = spark.read.option("header", True).csv(
    f"{base_path}/Camera_Traffic_Counts_20250125.csv"
)
for col_ in camera_traffic.columns:

    camera_traffic = camera_traffic.withColumnRenamed(col_, to_snake_case(col_))

camera_traffic.limit(10).show()

+--------------------+-------------+--------------------+------------------+----------+----------+-------------+------+----------------------------+-------------+-----------------------+-----------------------+-----+---+----+----+------+-----------+--------------------+
|           record_id|atd_device_id|           read_date| intersection_name| direction|  movement|heavy_vehicle|volume|speed_average_miles_per_hour|speed_std_dev|seconds_in_zone_average|seconds_in_zone_std_dev|month|day|year|hour|minute|day_of_week|bin_duration_seconds|
+--------------------+-------------+--------------------+------------------+----------+----------+-------------+------+----------------------------+-------------+-----------------------+-----------------------+-----+---+----+----+------+-----------+--------------------+
|f6d2caebed1dc902b...|         7047|03/11/2020 02:30:...|MANOR RD / 51ST ST|NORTHBOUND| LEFT TURN|        false|    14|                         8.5|         2.21|                 12.407| 

## Radar Traffic Counts

In [8]:
radar_traffic = spark.read.option("header", True).csv(
    f"{base_path}/Radar_Traffic_Counts_20250125.csv"
)
for col_ in radar_traffic.columns:
    radar_traffic = radar_traffic.withColumnRenamed(col_, to_snake_case(col_))
radar_traffic.limit(10).show()

+--------------------+-----------+-------+--------------------+--------------------+--------+------+---------+-----+-----+---+----+----+------+-----------+--------+---------+
|              row_id|detector_id|kits_id|           read_date|   intersection_name|    lane|volume|occupancy|speed|month|day|year|hour|minute|day_of_week|time_bin|direction|
+--------------------+-----------+-------+--------------------+--------------------+--------+------+---------+-----+-----+---+----+----+------+-----------+--------+---------+
|9794d3655e0572eb8...|         74|     19|01/24/2018 04:15:...|         KINNEYLAMAR|   Lane1|     4|        0|   18|    1| 23|2018|  22|    15|          2|   22:15|     None|
|1f51f4e6a68297c9f...|         92|     24|12/17/2017 01:45:...|    LOOP 360LAKEWOOD|  NB_out|   103|        6|   40|   12| 16|2017|  19|    45|          6|   19:45|       NB|
|9de27c4292306d305...|         10|      3|01/01/156489 12:0...|     LAMARSHOALCREEK|  SB_out|    20|        1|   33|    8|  3

In [10]:
radar_traffic.select("detector_id").distinct().count()

86

## Traffic Detectors

In [9]:
traffic_detectors = spark.read.option("header", True).csv(
    f"{base_path}/Traffic_Detectors_20250125.csv"
)
for col_ in traffic_detectors.columns:
    traffic_detectors = traffic_detectors.withColumnRenamed(col_, to_snake_case(col_))
traffic_detectors.limit(10).show()

+-----------+-------------+---------------+------------------+-----------------+--------------------+---------------+---------+--------------------+--------------------+--------------+------------------------+-----------------+------------------+--------------------+
|detector_id|detector_type|detector_status|detector_direction|detector_movement|       location_name|atd_location_id|signal_id|        created_date|       modified_date|ip_comm_status|comm_status_datetime_utc|location_latitude|location_longitude|            location|
+-----------+-------------+---------------+------------------+-----------------+--------------------+---------------+---------+--------------------+--------------------+--------------+------------------------+-----------------+------------------+--------------------+
|        257|        VIDEO|             OK|        NORTHBOUND|        THRU ONLY| 12700 BLK DESSAU RD|   LOC16-004035|      820|                NULL|10/07/2024 09:31:...|          NULL|            

## Individual Address Files

In [16]:
IAFs = spark.read.option("header", True).csv(
    f"{base_path}/Bluetooth_Travel_Sensors_-_Individual_Address_Files__IAFs_.csv"
)
for col_ in IAFs.columns:
    IAFs = IAFs.withColumnRenamed(col_, to_snake_case(col_))
IAFs.limit(10).show()

+--------------------+--------------------+----------------------+-----------------+--------------+
|           record_id|      host_read_time|field_device_read_time|reader_identifier|device_address|
+--------------------+--------------------+----------------------+-----------------+--------------+
|e60fa504f56592882...|08/28/2019 08:08:...|  08/28/2019 08:08:...|     lamar_morrow|          5444|
|6ccf9bc09f94adcf6...|08/28/2019 08:08:...|  08/28/2019 08:08:...|     lamar_morrow|         37197|
|8c92a42d206a0c867...|08/28/2019 08:08:...|  08/28/2019 08:08:...|     lamar_morrow|          7947|
|b141f5991a32e5874...|08/28/2019 08:09:...|  08/28/2019 08:09:...|     lamar_morrow|          1701|
|9a945c4a0553c1e4e...|08/28/2019 08:09:...|  08/28/2019 08:09:...|     lamar_morrow|           278|
|0c009feda4adb5440...|08/28/2019 08:09:...|  08/28/2019 08:09:...|     lamar_morrow|         37198|
|26ab5aaf898ed8da9...|08/28/2019 08:09:...|  08/28/2019 08:09:...|     lamar_morrow|         24651|


## Individual Traffic Match Files

In [15]:
ITMF = spark.read.option("header", True).csv(
    f"{base_path}/Bluetooth_Travel_Sensors_-_Individual_Traffic_Match_Files__ITMF_.csv"
)
for col_ in ITMF.columns:
    ITMF = ITMF.withColumnRenamed(col_, to_snake_case(col_))
ITMF.limit(10).show()

+--------------------+--------------+------------------------+-----------------------------+-------------------+--------------------+--------------+-----------------+-------------------+-------------------+-----------+
|           record_id|device_address|origin_reader_identifier|destination_reader_identifier|travel_time_seconds|speed_miles_per_hour|match_validity|filter_identifier|         start_time|           end_time|day_of_week|
+--------------------+--------------+------------------------+-----------------------------+-------------------+--------------------+--------------+-----------------+-------------------+-------------------+-----------+
|cdac7d191483cacf2...|           706|              lamar_45th|                   lamar_38th|                 50|                  47|         valid|              125|2019-08-17T23:28:56|2019-08-17T23:29:46|   Saturday|
|6e2a448e185b9742c...|           707|              lamar_45th|                   lamar_38th|                 58|            

## Traffic Match Summary Records

In [13]:
TMSR = spark.read.option("header", True).csv(
    f"{base_path}/Bluetooth_Travel_Sensors_-Traffic_Match_Summary_Records__TMSR__20250125.csv"
)
for col_ in TMSR.columns:
    TMSR = TMSR.withColumnRenamed(col_, to_snake_case(col_))
TMSR.limit(10).show()

+--------------------+------------------------+-----------------------------+--------------+-------------------+----------------+-------------------+------------------------+---------------------+--------------------+--------------------+---------------------------+-----------------+------------------------+--------------+------------------+
|           record_id|origin_reader_identifier|destination_reader_identifier|origin_roadway|origin_cross_street|origin_direction|destination_roadway|destination_cross_street|destination_direction|segment_length_miles|           timestamp|average_travel_time_seconds|average_speed_mph|summary_interval_minutes|number_samples|standard_deviation|
+--------------------+------------------------+-----------------------------+--------------+-------------------+----------------+-------------------+------------------------+---------------------+--------------------+--------------------+---------------------------+-----------------+------------------------+---

In [11]:
traffic_detectors.select("detector_id").distinct().count()

5384

In [ ]:
[
    "Record ID",
    "ATD Device ID",
    "Read Date",
    "Intersection Name",
    "Direction",
    "Movement",
    "Heavy Vehicle",
    "Volume",
    "Speed Average (Miles Per Hour)",
    "Speed StdDev",
    "Seconds in Zone Average",
    "Seconds in Zone StdDev",
    "Month",
    "Day",
    "Year",
    "Hour",
    "Minute",
    "Day of Week",
    "Bin Duration (Seconds)",
]

In [ ]:
[
    "record _i_d",
    "a_t_d _device _i_d",
    "read _date",
    "intersection _name",
    "direction",
    "movement",
    "heavy _vehicle",
    "volume",
    "speed _average (_miles _per _hour)",
    "speed _std_dev",
    "seconds in _zone _average",
    "seconds in _zone _std_dev",
    "month",
    "day",
    "year",
    "hour",
    "minute",
    "day of _week",
    "bin _duration (_seconds)",
]

In [11]:
spark.stop()